# Etape 1.5 : Essais de clustering

## Ces essais n'étaient pas concluant mais maritent tout de même d'être notés.

Import des librairies

In [ ]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, prince
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap

Import du dataset

In [ ]:
dataset = pd.read_csv('./Datasets/complete_dataset_one_hot_with_scaling.pkl')

## PCA

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(dataset.drop('Attrition', axis=1))
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, dataset[['Attrition']]], axis = 1)
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# Visualisation des données
sns.set_style('whitegrid')
sns.lmplot(x = 'principal component 1',y = 'principal component 2', data=finalDf, hue='Attrition', fit_reg=False, height=6, aspect=1.5)
plt.show()

## t-SNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(dataset.drop('Attrition', axis=1))
tsneDf = pd.DataFrame(data = tsne_results, columns = ['tsne component 1', 'tsne component 2'])
finalDf = pd.concat([tsneDf, dataset[['Attrition']]], axis = 1)

In [ ]:
# Visualisation des données
sns.set_style('whitegrid')
sns.lmplot(x='tsne component 1', y='tsne component 2', data=finalDf, hue='Attrition', fit_reg=False, height=6,
           aspect=1.5)
plt.show()

Essayons sans les valeurs catégoriques

## UMAP

In [ ]:
reducer = umap.UMAP()
umap_results = reducer.fit_transform(dataset.drop('Attrition', axis=1))
umapDf = pd.DataFrame(data = umap_results, columns = ['umap component 1', 'umap component 2'])
finalDf = pd.concat([umapDf, dataset[['Attrition']]], axis = 1)

In [ ]:
# Visualisation des données
sns.set_style('whitegrid')
sns.lmplot(x = 'umap component 1',y = 'umap component 2', data=finalDf, hue='Attrition', fit_reg=False, height=6, aspect=1.5)
plt.show()

## Choix du bon clustering : MCA

https://maxhalford.github.io/prince/mca/


In [ ]:
dataset = pd.read_csv('./Datasets/complete_dataset_with_scaling.pkl')
# drop attrition
dataset = dataset.drop('Attrition', axis=1)
dataset.columns = ['BusinessTravel', 'Department', 'DistanceFromHome',
                   'Education', 'EducationField', 'JobLevel', 'JobRole', 'MonthlyIncome',
                   'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
                   'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
                   'YearsSinceLastPromotion', 'YearsWithCurrManager', 'hrs',
                   'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
                   'JobSatisfaction', 'WorkLifeBalance']

# Récupération des colonnes catégorielles (sauf "Attrition")
cat_cols = dataset.select_dtypes(exclude=['int64', 'float64']).columns
# Génération des colonnes one-hot encoding
dummy_df = pd.get_dummies(dataset[cat_cols], drop_first=True)
# Concaténation des colonnes one-hot encoding avec le dataset complet
dataset = pd.concat([dataset, dummy_df], axis=1)
# Suppression des colonnes catégorielles (sauf "Attrition")
dataset.drop(columns=cat_cols, inplace=True)

mca = prince.MCA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mca = mca.fit(dataset)

mca.eigenvalues_summary

In [ ]:
mca.row_coordinates(dataset).head()

In [ ]:
mca.column_coordinates(dataset).head()

In [ ]:
mca.plot(
    dataset,
    x_component=0,
    y_component=1
)

In [ ]:
mca.row_contributions_.head().style.format('{:.0%}')

In [ ]:
mca.column_contributions_.head().style.format('{:.0%}')